In [15]:
from pathlib import Path
import sys
sys.path.append('../../30_data_tools/')
from helper import load_dotenv
from tqdm import tqdm

In [6]:
import pandas as pd
import sqlite3
from mask_functions import get_config

In [13]:
import shutil

In [7]:
config = get_config()
config['target_variant'] = 'halftone600dpi'

In [8]:
dotenv = load_dotenv()

In [9]:
con = sqlite3.connect(dotenv['DB_PATH'])

In [10]:
remaining_pages = pd.read_sql(
    f'''
        SELECT cf.* FROM (
        	SELECT * FROM related_file
        	WHERE variant_name = '{ config["target_variant"] }' AND "type" = '4c'
        ) cf
        LEFT JOIN (
        	SELECT job, pdf_filename, 1 AS has_mask FROM related_file 
        	WHERE variant_name = '{ config["target_variant"] }' AND "type" = 'masks'
        ) mf ON cf.job=mf.job AND cf.pdf_filename=mf.pdf_filename 
        WHERE mf.has_mask IS NULL
    ''',
    con
).sample(frac=1)

In [12]:
OUT_DIR = Path('/Volumes/Hotfolder/Switch/testing/birw001_masterarbeit_moireerkennung/render_masks/in/')

In [17]:
for i in tqdm(range(remaining_pages[:5].shape[0])):
    row = remaining_pages.iloc[i]
    img_path = dotenv['DATA_DIR'] / row['job'] / row['variant_name'] / row['filename']    
    mask_path_300dpi = dotenv['DATA_DIR'] / row['job'] / 'halftone300dpi' / f'{ img_path.name.strip( "." + row["type"] + img_path.suffix ) }.masks.pkl'

    if mask_path_300dpi.exists() == False:
        copy_target_path = OUT_DIR / f"{ row['job'] }.{ row['filename']   }"
        
        shutil.copy(
            img_path,
            copy_target_path
        )

100%|███████████████████████████████████| 5/5 [00:09<00:00,  1.95s/it]
